In [18]:
#import os
import numpy as np
import pandas as pd
#import collections
#import matplotlib.pyplot as plt
import networkx as nx
import math
import re
#from difflib import SequenceMatcher
from tqdm import tqdm, tnrange, tqdm_notebook
import mmap
import random

In [19]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11195956107030461792
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2201773105136009482
physical_device_desc: "device: XLA_CPU device"
]


In [20]:
def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [21]:
emb = []
with open('word2vec_output.txt', 'r') as f_emb:
    for line in tqdm(f_emb, total=get_num_lines('word2vec_output.txt')):
        emb.append(line.split(' '))
f_emb.close()

100%|██████████| 1559889/1559889 [00:30<00:00, 51288.59it/s]


In [22]:
for i in tqdm(range(len(emb))):
    emb[i][len(emb[i])-1] = emb[i][len(emb[i])-1][:-1]

100%|██████████| 1559889/1559889 [00:01<00:00, 1103742.28it/s]


In [23]:
emb_dict = {}
for i in tqdm(range(1,len(emb))):
    emb_dict[int(emb[i][0])] = [float(j) for j in emb[i][1:]]

100%|██████████| 1559888/1559888 [00:49<00:00, 31682.10it/s]


In [24]:
G = nx.Graph() 
with open('AMiner-Coauthor.txt', 'r') as file:
    for line in tqdm(file, total=get_num_lines('AMiner-Coauthor.txt')):
        line_arr = line.split()
        G.add_edge(int(line_arr[0][1:]), int(line_arr[1]), weight = int(line_arr[2]))

100%|██████████| 4258946/4258946 [00:23<00:00, 184476.44it/s]


In [25]:
Edges_pos = list(G.edges())

In [26]:
G_agj = G.adjacency()

In [27]:
nodes_AMiner = np.array(G.nodes())

In [28]:
missed = set(nodes_AMiner)-set(emb_dict)
list(missed)[:5]

[88064, 593920, 1341443, 1503235, 1417221]

In [29]:
num_e = G.number_of_edges()
num_n = G.number_of_nodes()

In [30]:
num_e

4258946

In [31]:
num_n

1560640

In [32]:
edges_pos = []
for e in Edges_pos:
    if e[0] not in missed and e[1] not in missed:
        edges_pos.append(e)

In [17]:
edges_AMiner_neg = []
kol_e = 0
N = int(math.ceil(num_e/float(num_n)))
for i in G_agj:
    k = 0
    while k != N:
        j = random.choice(nodes_AMiner)
        if j not in i[1].keys():
            edges_AMiner_neg.append((i[0],j))
            k += 1
            kol_e += 1
            if kol_e == num_e:
                break 

In [18]:
# Запись
Edges_neg = []      
fw_neg = open('AMiner-Coauthor_neg2.txt', 'w')                                
for e in edges_AMiner_neg:
    fw_neg.write(str(e[0])+" "+str(e[1])+'\n')
    Edges_neg.append((int(e[0]),int(e[1])))
fw_neg.close()

In [33]:
# Считывание
Edges_neg = []    
with open('AMiner-Coauthor_neg2.txt', 'r') as file:
    for line in tqdm(file, total=get_num_lines('AMiner-Coauthor_neg2.txt')):
        e = line.split()
        Edges_neg.append((int(e[0]),int(e[1])))
file.close()

100%|██████████| 4681919/4681919 [00:04<00:00, 999423.00it/s] 


In [34]:
edges_neg = []
for e in Edges_neg:
    if e[0] not in missed and e[1] not in missed:
        edges_neg.append(e)

In [35]:
len(edges_pos)

4258570

In [36]:
len(edges_neg)

4677469

In [37]:
def model_svm(X_train, y_train, X_test, y_test):
    
    # SVC
    svm_model = svm.SVC(probability=True)
    svm_model.fit(X_train, y_train)
                     
    y_pred_train_res = svm_model.predict_proba(X_train)#svm_model.predict_proba(X_train)
    y_pred_test_res = svm_model.predict_proba(X_test)#svm_model.predict_proba(X_test)
    y_pred_train = [np.argmax(y_pred_train_res[i]) for i in range(len(y_pred_train_res))]
    y_pred_test = [np.argmax(y_pred_test_res[i]) for i in range(len(y_pred_test_res))]
    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test

In [38]:
def model_logistic(X_train, y_train, X_test, y_test):
    
    # Logistic regression
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train, y_train)
                     
    y_pred_train_res = logistic_model.predict_proba(X_train)
    y_pred_test_res = logistic_model.predict_proba(X_test)
    y_pred_train = [np.argmax(y_pred_train_res[i]) for i in range(len(y_pred_train_res))]
    y_pred_test = [np.argmax(y_pred_test_res[i]) for i in range(len(y_pred_test_res))]
    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test

In [39]:
def model_rf(X_train, y_train, X_test, y_test):
    
    # RandomForest
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
                     
    y_pred_train_res = rf.predict_proba(X_train)
    y_pred_test_res = rf.predict_proba(X_test)
    y_pred_train = [np.argmax(y_pred_train_res[i]) for i in range(len(y_pred_train_res))]
    y_pred_test = [np.argmax(y_pred_test_res[i]) for i in range(len(y_pred_test_res))]
    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test

In [40]:
def model_gbc(X_train, y_train, X_test, y_test):
    
    # Gradient Boosting
    params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
              'learning_rate': 0.01, 'loss': 'ls'}
    clf = GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)
                     
    y_pred_train_res = clf.predict_proba(X_train)
    y_pred_test_res = clf.predict_proba(X_test)
    y_pred_train = [np.argmax(y_pred_train_res[i]) for i in range(len(y_pred_train_res))]
    y_pred_test = [np.argmax(y_pred_test_res[i]) for i in range(len(y_pred_test_res))]
    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test

In [41]:
def models(X_train, y_train, X_test, y_test):
    # LogReg
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train,f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_logistic(X_train, y_train, X_test, y_test)
    print("Logistic regression")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))
    # SVM
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_svm(X_train, y_train, X_test, y_test)
    print("SVM")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))
    # RF
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train,f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_rf(X_train, y_train, X_test, y_test)
    print("Random Forest")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))
    # GBC
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train,f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_gbc(X_train, y_train, X_test, y_test)
    print("Gradient Boosting")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))

In [42]:
# Define edge embbeding functions
def avg_sum(v1, v2):
    return (np.array(v1)+np.array(v2))/2

def mult(v1, v2):
    return np.array(v1)*np.array(v2)

def w_l1(v1, v2):
    return np.abs(np.array(v1)-np.array(v2))

def w_l2(v1, v2):
    return (np.array(v1)-np.array(v2))**2

def nw_l1(v1, v2, graph, n1, n2, embs ):
    #neig1 = [n for n in graph.neighbors(n1)]
    #neig2 = [n for n in graph.neighbors(n2)]
    sum1 = np.zeros(len(v1))
    sum2 = np.zeros(len(v2))
    for n in graph.neighbors(n1):
        sum1 += np.array(embs[int(n)])
    for n in graph.neighbors(n2):
        sum2 += np.array(embs[int(n)])
    return np.abs((sum1+np.array(v1))/(graph.degree(n1)+1)-(sum2+np.array(v2))/(graph.degree(n2)+1))

def nw_l2(v1, v2, graph, n1, n2, embs ):
    #neig1 = [n for n in graph.neighbors(n1)]
    #neig2 = [n for n in graph.neighbors(n2)]
    sum1 = np.zeros(len(v1))
    sum2 = np.zeros(len(v2))
    for n in graph.neighbors(n1):
        sum1 += np.array(embs[int(n)])
    for n in graph.neighbors(n2):
        sum2 += np.array(embs[int(n)])
    return ((sum1+np.array(v1))/(graph.degree(n1)+1)-(sum2+np.array(v2))/(graph.degree(n2)+1))**2

In [43]:
import numpy as np
import collections
import os
import pandas as pd
import networkx as nx
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

In [33]:
%%time
## 6: avg_sum ##
X_train = []
y_train = []
X_test = []
y_test = []
num_train = int(round(.8*len(edges_pos)))
m = 0
for e in edges_pos[:num_train]:
        emb1 = emb_dict[int(e[0])]
        emb2 = emb_dict[int(e[1])]
        w = 1 #G_sub20[int(e[0])][int(e[1])]['weight']
        y_train.append(w)
        res = avg_sum(emb1, emb2)
        X_train.append(res)
#
for e in edges_pos[num_train:]:
        emb1 = emb_dict[int(e[0])]
        emb2 = emb_dict[int(e[1])]
        w = 1 
        y_test.append(w)
        res = avg_sum(emb1, emb2)
        X_test.append(res)
#
for e in edges_neg[:num_train]:
        emb1 = emb_dict[int(e[0])]
        emb2 = emb_dict[int(e[1])]
        w = 0 
        y_train.append(w)
        res = avg_sum(emb1, emb2)
        X_train.append(res)
#
for e in edges_neg[num_train:]:
        emb1 = emb_dict[int(e[0])]
        emb2 = emb_dict[int(e[1])]
        w = 0 
        y_test.append(w)
        res = avg_sum(emb1, emb2)
        X_test.append(res)

CPU times: user 1min 38s, sys: 386 ms, total: 1min 38s
Wall time: 1min 38s


In [34]:
    %%time
    # LogReg
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train,f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_logistic(X_train, y_train, X_test, y_test)
    print("Logistic regression (avg_sum): test")
    print('Precision: ' + str(precision_test))
    print('Accuracy: ' + str(accuracy_test))
    print('F-1 (macro): ' + str(f1_macro_test))
    print('F-1 (micro): ' + str(f1_micro_test))
    print('Logloss: ' + str(logloss_test))
    print('ROC-AUC: ' + str(roc_auc_test))
    print("Logistic regression (avg_sum): train")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))

/home/olga3993/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression (avg_sum): test
Precision: 0.5141144414168938
Accuracy: 0.6040581870748476
F-1 (macro): 0.5247652242339096
F-1 (micro): 0.6040581870748476
Logloss: 13.675419643275355
ROC-AUC: 0.5446534292916163
Logistic regression (avg_sum): train
Precision: 0.7449377504293074
Accuracy: 0.737135646472877
F-1 (macro): 0.7370689513775414
F-1 (micro): 0.737135646472877
Logloss: 9.079111854019526
ROC-AUC: 0.7371356464728771
CPU times: user 2min 41s, sys: 9.07 s, total: 2min 50s
Wall time: 2min 43s


In [57]:
%%time
## 5: mult ##
X_train = []
y_train = []
X_test = []
y_test = []
num_train = int(round(.00001*len(edges_pos)))
for e in edges_pos[:num_train]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 1 #G_sub20[int(e[0])][int(e[1])]['weight']
    y_train.append(w)
    res = mult(emb1, emb2)
    X_train.append(res)
#
for e in edges_pos[num_train:]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 1 
    y_test.append(w)
    res = mult(emb1, emb2)
    X_test.append(res)
#
for e in edges_neg[:num_train]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 0 
    y_train.append(w)
    res = mult(emb1, emb2)
    X_train.append(res)
#
for e in edges_neg[num_train:]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 0 
    y_test.append(w)
    res = mult(emb1, emb2)
    X_test.append(res)

CPU times: user 1min 22s, sys: 352 ms, total: 1min 23s
Wall time: 1min 23s


In [58]:
    %%time
    # LogReg
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train,f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_logistic(X_train, y_train, X_test, y_test)
    print("Logistic regression (mult): test")
    print('Precision: ' + str(precision_test))
    print('Accuracy: ' + str(accuracy_test))
    print('F-1 (macro): ' + str(f1_macro_test))
    print('F-1 (micro): ' + str(f1_micro_test))
    print('Logloss: ' + str(logloss_test))
    print('ROC-AUC: ' + str(roc_auc_test))
    print("Logistic regression (mult): train")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))

/home/olga3993/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression (mult): test
Precision: 0.9960359759584404
Accuracy: 0.6519629187843758
F-1 (macro): 0.5880497920028142
F-1 (micro): 0.6519629187843758
Logloss: 12.020775335871873
ROC-AUC: 0.6348934713553004
Logistic regression (mult): train
Precision: 1.0
Accuracy: 1.0
F-1 (macro): 1.0
F-1 (micro): 1.0
Logloss: 9.992007221626413e-16
ROC-AUC: 1.0
CPU times: user 48.6 s, sys: 5.47 s, total: 54.1 s
Wall time: 46.8 s


In [59]:
    # Logistic regression
    logistic_model = LogisticRegression()
    logistic_model.fit(X_train, y_train)
                     
    y_pred_train_res = logistic_model.predict_proba(X_train)
    y_pred_test_res = logistic_model.predict_proba(X_test)
    y_pred_train = [np.argmax(y_pred_train_res[i]) for i in range(len(y_pred_train_res))]
    y_pred_test = [np.argmax(y_pred_test_res[i]) for i in range(len(y_pred_test_res))]



/home/olga3993/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [60]:
logistic_model.coef_

array([[-0.0300981 , -0.06513173, -0.319959  ,  0.06301884,  0.1433852 ,
        -0.24352267,  0.10440547,  0.35089039,  0.5177037 ,  0.11181678,
         0.19231446, -0.02313459, -0.13008327,  0.01908783,  0.57745613,
         0.15476395,  0.29231465,  0.01416023, -0.19845081,  0.16711637,
        -0.33290153,  0.2176758 ,  0.33928428,  0.0624466 ,  0.48898385,
         0.09226164,  0.58650877,  0.12591223, -0.16382232,  0.41285882,
         0.70424886,  0.34348407,  0.08193557,  0.73959105, -0.16329023,
        -0.27212709,  0.31208437,  0.51141764, -0.0050305 ,  0.11954339,
         0.25752024, -0.08659142,  0.01123745, -0.01286996,  0.56364143,
        -0.13980124,  0.28986611, -0.31313673,  0.56979489,  0.0540012 ,
         1.03165817,  0.37036491,  0.12126538,  0.19774262,  1.09156032,
         0.06793979,  0.15267238, -0.02914582,  0.00293731,  0.08176616,
        -0.07033101,  0.30985919,  0.10954587,  0.07466707,  0.06159794,
        -0.09609363, -0.05447392, -0.05424918,  0.0

In [61]:
print (len(y_pred_test))
print (sum(y_pred_test))

8935953
1157662


In [62]:
print (len(y_pred_train))
print (sum(y_pred_train))

86
43


In [37]:
%%time
## 4: w_l1 ##
X_train = []
y_train = []
X_test = []
y_test = []
num_train = int(round(.8*len(edges_pos)))
for e in edges_pos[:num_train]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 1 #G_sub20[int(e[0])][int(e[1])]['weight']
    y_train.append(w)
    res = w_l1(emb1, emb2)
    X_train.append(res)
#
for e in edges_pos[num_train:]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 1 
    y_test.append(w)
    res = w_l1(emb1, emb2)
    X_test.append(res)
#
for e in edges_neg[:num_train]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 0 
    y_train.append(w)
    res = w_l1(emb1, emb2)
    X_train.append(res)
#
for e in edges_neg[num_train:]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 0 
    y_test.append(w)
    res = w_l1(emb1, emb2)
    X_test.append(res)

CPU times: user 1min 28s, sys: 461 ms, total: 1min 29s
Wall time: 1min 29s


In [38]:
    %%time
    # LogReg
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train,f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_logistic(X_train, y_train, X_test, y_test)
    print("Logistic regression (w_l1): test")
    print('Precision: ' + str(precision_test))
    print('Accuracy: ' + str(accuracy_test))
    print('F-1 (macro): ' + str(f1_macro_test))
    print('F-1 (micro): ' + str(f1_micro_test))
    print('Logloss: ' + str(logloss_test))
    print('ROC-AUC: ' + str(roc_auc_test))
    print("Logistic regression (w_l1): train")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))

/home/olga3993/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression (w_l1): test
Precision: 0.7389987293697482
Accuracy: 0.856237516650356
F-1 (macro): 0.8557201762127032
F-1 (micro): 0.856237516650356
Logloss: 4.965492713458297
ROC-AUC: 0.8786486595054152
Logistic regression (w_l1): train
Precision: 0.8406465788758758
Accuracy: 0.8349246343256069
F-1 (macro): 0.834912989522812
F-1 (micro): 0.8349246343256069
Logloss: 5.701563782484556
ROC-AUC: 0.8349246343256069
CPU times: user 1min 52s, sys: 3.58 s, total: 1min 55s
Wall time: 1min 48s
Parser   : 211 ms


In [39]:
%%time
X_train2 =[]
for x in X_train:
    X_train2.append(list(np.array(x)**2))

X_test2 =[]
for x in X_test:
    X_test2.append(list(np.array(x)**2))

CPU times: user 2min 12s, sys: 8.75 s, total: 2min 21s
Wall time: 2min 20s


In [40]:
    %%time
    # LogReg
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train,f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_logistic(X_train2, y_train, X_test2, y_test)
    print("Logistic regression (w_l2): test")
    print('Precision: ' + str(precision_test))
    print('Accuracy: ' + str(accuracy_test))
    print('F-1 (macro): ' + str(f1_macro_test))
    print('F-1 (micro): ' + str(f1_micro_test))
    print('Logloss: ' + str(logloss_test))
    print('ROC-AUC: ' + str(roc_auc_test))
    print("Logistic regression (w_l2): train")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))

/home/olga3993/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression (w_l2): test
Precision: 0.7069161371525676
Accuracy: 0.8323081221696751
F-1 (macro): 0.8321121950623813
F-1 (micro): 0.8323081221696751
Logloss: 5.79200456803233
ROC-AUC: 0.8590313512818832
Logistic regression (w_l2): train
Precision: 0.8205244715921372
Accuracy: 0.8321092819890245
F-1 (macro): 0.8320544335809745
F-1 (micro): 0.8321092819890246
Logloss: 5.798814315676781
ROC-AUC: 0.8321092819890245
CPU times: user 3min 38s, sys: 6.65 s, total: 3min 45s
Wall time: 3min 37s


In [44]:
%%time
## 2: nw_l1 ##
X_train = []
y_train = []
X_test = []
y_test = []
num_train = int(round(.4*len(edges_pos)))
for e in edges_pos[:num_train]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 1 #G_sub20[int(e[0])][int(e[1])]['weight']
    y_train.append(w)
    res = nw_l1(emb1, emb2, G,int(e[0]),int(e[1]),emb_dict)
    X_train.append(res)
#
for e in edges_pos[num_train:]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 1 
    y_test.append(w)
    res = nw_l1(emb1, emb2, G,int(e[0]),int(e[1]),emb_dict)
    X_test.append(res)
#
for e in edges_neg[:num_train]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 0 
    y_train.append(w)
    res = nw_l1(emb1, emb2, G,int(e[0]),int(e[1]),emb_dict)
    X_train.append(res)
#
for e in edges_neg[num_train:]:
    emb1 = emb_dict[int(e[0])]
    emb2 = emb_dict[int(e[1])]
    w = 0 
    y_test.append(w)
    res = nw_l1(emb1, emb2, G,int(e[0]),int(e[1]),emb_dict)
    X_test.append(res) 

CPU times: user 25min 13s, sys: 24.4 s, total: 25min 37s
Wall time: 25min 2s


In [45]:
    %%time
    # LogReg
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train,f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_logistic(X_train, y_train, X_test, y_test)
    print("Logistic regression (nw_l1): test")
    print('Precision: ' + str(precision_test))
    print('Accuracy: ' + str(accuracy_test))
    print('F-1 (macro): ' + str(f1_macro_test))
    print('F-1 (micro): ' + str(f1_micro_test))
    print('Logloss: ' + str(logloss_test))
    print('ROC-AUC: ' + str(roc_auc_test))
    print("Logistic regression (nw_l1): train")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))

/home/olga3993/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression (nw_l1): test
Precision: 0.9851604618531395
Accuracy: 0.9920069565431276
F-1 (macro): 0.9919677510736085
F-1 (micro): 0.9920069565431276
Logloss: 0.2760754939950439
ROC-AUC: 0.9924101771268805
Logistic regression (nw_l1): train
Precision: 0.9926552585135976
Accuracy: 0.9914645643960297
F-1 (macro): 0.9914645519313676
F-1 (micro): 0.9914645643960297
Logloss: 0.2948064310798988
ROC-AUC: 0.9914645643960297
CPU times: user 1min 20s, sys: 8.22 s, total: 1min 29s
Wall time: 1min 21s


In [46]:
%%time
X_train2 =[]
for x in X_train:
    X_train2.append(list(np.array(x)**2))

X_test2 =[]
for x in X_test:
    X_test2.append(list(np.array(x)**2))

CPU times: user 2min 18s, sys: 9.99 s, total: 2min 28s
Wall time: 2min 27s


In [47]:
    %%time
    # LogReg
    precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train,f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train,roc_auc_test, y_pred_train, y_pred_test  = model_logistic(X_train2, y_train, X_test2, y_test)
    print("Logistic regression (nw_l2): test")
    print('Precision: ' + str(precision_test))
    print('Accuracy: ' + str(accuracy_test))
    print('F-1 (macro): ' + str(f1_macro_test))
    print('F-1 (micro): ' + str(f1_micro_test))
    print('Logloss: ' + str(logloss_test))
    print('ROC-AUC: ' + str(roc_auc_test))
    print("Logistic regression (nw_l2): train")
    print('Precision: ' + str(precision_train))
    print('Accuracy: ' + str(accuracy_train))
    print('F-1 (macro): ' + str(f1_macro_train))
    print('F-1 (micro): ' + str(f1_micro_train))
    print('Logloss: ' + str(logloss_train))
    print('ROC-AUC: ' + str(roc_auc_train))

/home/olga3993/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression (nw_l2): test
Precision: 0.984098067256659
Accuracy: 0.9916598889926415
F-1 (macro): 0.9916197641916635
F-1 (micro): 0.9916598889926415
Logloss: 0.28806318859140984
ROC-AUC: 0.9921120551101485
Logistic regression (nw_l2): train
Precision: 0.9919908634776916
Accuracy: 0.9918831321312084
F-1 (macro): 0.9918831320339117
F-1 (micro): 0.9918831321312084
Logloss: 0.2803498856885695
ROC-AUC: 0.9918831321312084
CPU times: user 2min 49s, sys: 8.33 s, total: 2min 58s
Wall time: 2min 50s


In [65]:
%%time
f = open('X_train_nw_l2', 'w')                                
for x in tqdm(X_train):
    line = ""
    for i in x:
        line += str(i)
        line += " "
    f.write(line+'\n')
f.close()

100%|██████████| 6813712/6813712 [56:12<00:00, 2020.56it/s]  

CPU times: user 39min 10s, sys: 33.3 s, total: 39min 43s
Wall time: 56min 12s


In [64]:
f = open('X_test_nw_l2', 'w')                                
for x in tqdm(X_test):
    line = ""
    for i in x:
        line += str(i)
        line += " "
    f.write(line+'\n')
f.close()

100%|██████████| 2122304/2122304 [16:22<00:00, 2161.12it/s]


In [62]:
f = open('y_train_nw_l2', 'w')                                
for y in tqdm(y_train):
    line = str(y)
    f.write(line+'\n')
f.close()

f = open('y_test_nw_l2', 'w')                                
for y in tqdm(y_test):
    line = str(y)
    f.write(line+'\n')
f.close()

100%|██████████| 2122304/2122304 [00:04<00:00, 521256.21it/s]
